In [1]:
import numpy as np
import operator

In [2]:
supply = np.array([150,160,90])
demand = np.array([140,120,90,50])
costs = np.array([[16,18,21,12],[17,19,14,13],[32,11,15,10]])

n,m = costs.shape

In [3]:
X = np.full((n, m), np.nan)
allow_fill_X = np.ones((n, m), dtype=bool)
indices = [(i, j) for i in range(n) for j in range(m)]

In [4]:
def _fill_zero_indice(i, j): 
    allow_fill_X[i, j] = False
    allowed_indices_i = [(i, jj) for jj in range(m) if allow_fill_X[i, jj]]
    allowed_indices_j = [(ii, j) for ii in range(n) if allow_fill_X[ii, j]]
    allowed_indices = allowed_indices_i + allowed_indices_j
    if allowed_indices:
        return allowed_indices[0]
    else:
        return None

In [7]:
C = np.copy(costs)
s = np.copy(supply)
d = np.copy(demand)

In [9]:
n_iter = 0
while n_iter < m + n - 1:
    row_diff = np.array([np.nan]*n)
    col_diff = np.array([np.nan]*m)
    for i in range(n):
        row_allowed = []
        for j in range(m):
            if allow_fill_X[i, j]:
                row_allowed.append(C[i, j])
        row_allowed_sorted = sorted(row_allowed)
        try:
            row_diff[i] = abs(row_allowed_sorted[0] - row_allowed_sorted[1])
        except:
            row_diff[i] = np.nan
    for j in range(m):
        col_allowed = []
        for i in range(n):
            if allow_fill_X[i, j]:
                col_allowed.append(C[i, j])
        col_allowed_sorted = sorted(col_allowed)
        try:
            col_diff[j] = abs(col_allowed_sorted[0] - col_allowed_sorted[1])
        except:
            col_diff[j] = np.nan

    try:
        diff = np.concatenate((row_diff, col_diff))
        max_diff_index = np.nanargmax(diff)
        max_diff = diff[max_diff_index]
    except:
        max_diff = None
    
    if max_diff:
        located = False
        while not located:
            for i in range(n):
                if row_diff[i] == max_diff:
                    located = True
                    located_type = "row"
                    located_index = i
                    break
            for j in range(m):
                if col_diff[j] == max_diff:
                    located = True
                    located_type = "col"
                    located_index = j
                    break

        if located_type == "row":
            row_indices = [(located_index, j) for j in range(m) if allow_fill_X[located_index, j]]
            row_values = [C[located_index,j] for j in range(m) if allow_fill_X[located_index, j]]
            xs = sorted(zip(row_indices, row_values), key=operator.itemgetter(1))
        else:
            col_indices = [(i, located_index) for i in range(n) if allow_fill_X[i, located_index]]
            col_values = [C[i, located_index] for i in range(n) if allow_fill_X[i, located_index]]
            xs = sorted(zip(col_indices, col_values), key=operator.itemgetter(1))

        (i, j), _ = xs[0]
    else:
        xs = [(i, j) for i in range(n) for j in range(m) if allow_fill_X[i, j]]
        (i, j) = xs[0]
            
    grabbed = min([s[i], d[j]])
    X[i, j] = grabbed
    if s[i] == grabbed and d[j] == grabbed:
        fill_zero_indices = _fill_zero_indice(i, j)
        if fill_zero_indices:
            X[fill_zero_indices] = 0
            allow_fill_X[fill_zero_indices] = False
            n_iter += 1

    s[i] -= grabbed
    d[j] -= grabbed

    if d[j] == 0:
        allow_fill_X[:, j] = False
    if s[i] == 0:
        allow_fill_X[i, :] = False

    n_iter += 1

In [10]:
for idx in range(3):
    print(costs[idx,:],supply[idx])

print(demand)
print(X)

[16 18 21 12] 150
[17 19 14 13] 160
[32 11 15 10] 90
[140 120  90  50]
[[100.  nan  nan  50.]
 [ 40.  30.  90.  nan]
 [ nan  90.  nan  nan]]


In [11]:
X_final = np.copy(X)
for i in range(0, n):
    for j in range(0,m):
        if np.isnan(X_final[i, j]):
            X_final[i, j] = 0
            
sumT = np.sum(X_final*C)
print(sumT)

5700.0
